## Training a Convolutional Neural Network (CNN) on MNIST Dataset with different weight initialization techniques

We begin with building a CNN network for image classification task on MNIST dataset. In the first part of the tutorial, we will try to understand the MNIST digit classfication dataset. Then we'll understand how to build a CNN network, define an appropiate loss, and train the network with back-propagation. Finally, we will use the trained model to test on a test dataset.

**Step 1:** We need to import all the required libraries

In [ ]:
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

import numpy as np
import matplotlib.pyplot as plt


cuda = torch.cuda.is_available()
print('Using PyTorch version:', torch.__version__, 'CUDA:', cuda)

**Step 2:** We need to load the MNIST data. In this tutorial, we'll be using Pytorch's dataloader to load and format the MNIST dataset. First time we may have to download the dataset, which may take a while. 

For more details, please refer: 
- https://pytorch.org/docs/0.3.0/data.html?highlight=dataloader#torch.utils.data.DataLoader
- https://pytorch.org/docs/0.3.0/torchvision/datasets.html?highlight=dataset

In [ ]:
batch_size = 10 # no. of examples to be processed at once on GPU

# Define the dataloader
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/tmp/step/data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,)),
                       
                   ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/tmp/step/data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=False)

**Step 3:** Visualize the dataset

- MNIST is a simple computer vision dataset. It consists of 28x28 pixel images of handwritten digits
- The train and test data are provided via data loaders that provide iterators over the datasets. 
- The first element of training data (X_train) is a 4th-order tensor of size (batch_size, 1, 28, 28), i.e. it consists of a batch of images of size 1x28x28 pixels. 
- y_train is a vector containing the correct classes ("0", "1", ..., "9") for each training digit.

For more visualizations and to get more insight on the MNIST dataset you may refer to this link: http://colah.github.io/posts/2014-10-Visualizing-MNIST/

In [ ]:
print("No. of Training examples:", len(train_loader)*batch_size)
print("No. of Testing examples:", len(test_loader)*batch_size)
for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

Here are the first 10 digits

In [ ]:
pltsize=1
plt.figure(figsize=(10*pltsize, pltsize))

for i in range(10):
    plt.subplot(1,10,i+1)
    plt.axis('off')
    plt.imshow(X_train[i,:,:,:].numpy().reshape(28,28), cmap="gray")
    plt.gca().set_title('\n Class: '+str(y_train[i].data[0].numpy()))
    

**Step 3:** Define the MLP network

![alt text](mnist_lenet.jpg "Logo Title Text 1")


- Here, we define a single layer MLP network (you are free to add few more layers)
- We have to write the \__init__() and forward() methods
- Pytorch will automatically generate the gradients when we call the backward() method for backward pass


### Weight Initialization

Try to initialize weights using different techniques given in the question papers. For your convenience, we have shown an example where we initialize all the weights to zero.

Observe the training and testing performance. Can you draw some conclusion?

#### Reference:
https://pytorch.org/docs/stable/nn.html#torch-nn-init


In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        # weight initialization code goes here
        nn.init.constant_(self.conv1.weight, 0)
        self.conv2 = nn.Conv2d(20, 50, 5)
        # weight initialization code goes here
        nn.init.constant_(self.conv2.weight, 0)
        self.fc1   = nn.Linear(50*4*4, 500)
        # weight initialization code goes here
        nn.init.constant_(self.fc1.weight, 0)
        self.fc2   = nn.Linear(500, 10)
        # weight initialization code goes here
        nn.init.constant_(self.fc2.weight, 0)
        

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        
        return out

model = LeNet()
if cuda:
    model.cuda()

**Step 4:** Define an optimizer
You can change the optimizers. Refer to the following links to know more about the available optimizers- https://pytorch.org/docs/0.3.0/optim.html#algorithms.

Here, we are using Stochastic Gradient Descent (SGD)


In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

**Step 5:** Define a suitable loss function.

    We will be using the Cross-Entropy loss function.     
    https://pytorch.org/docs/0.3.0/nn.html?highlight=cross%20entropy#torch.nn.functional.cross_entropy

In [ ]:
loss_func = nn.CrossEntropyLoss()

**Step 6:** Training method. We will use this method to train our model

In [ ]:
def train(epoch, log_interval=1000):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [ ]:
def test(loss_vector, accuracy_vector):
    model.eval()
    val_loss, correct = 0, 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        val_loss += loss_func(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    val_loss /= len(test_loader)
    loss_vector.append(val_loss)

    accuracy = 100. * correct / len(test_loader.dataset)
    accuracy_vector.append(accuracy)
    
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        val_loss, correct, len(test_loader.dataset), accuracy))

**Step 7:** Now, we are ready to train our first ever modelusing the train() function. 

Epoch: An epoch means  one pass through the whole training data. After each epoch, we'll evaluate our model on the test dataset.

In [ ]:
%%time
epochs = 10

lossv, accv = [], [] # used to keep track of train progress (used in the next cell)
for epoch in range(1, epochs + 1):
    train(epoch)
    test(lossv, accv)


**Step 8:** Now, as training is done, let us visualizae the training progress graphically

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(np.arange(1,epochs+1), lossv)
plt.title('validation loss')

plt.figure(figsize=(5,3))
plt.plot(np.arange(1,epochs+1), accv)
plt.title('validation accuracy');


In [ ]:
pltsize=1
plt.figure(figsize=(10*pltsize, pltsize))

X_test, y_test = iter(test_loader).next()
predicted_labels = model(Variable(X_test.cuda()))
                         
for i in range(10):
    plt.subplot(1,10,i+1)
    plt.axis('off')
    plt.imshow(X_test[i,:,:,:].numpy().reshape(28,28), cmap="gray")
    plt.gca().set_title('\n Class: '+str(predicted_labels.cpu().data.max(1)[1][i].data.numpy()))